In [1]:

import pandas as pd
import numpy as np
import folium
import geopandas
import geopy
from folium.plugins.marker_cluster import MarkerCluster
import folium.plugins as plugins
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [5]:
# Test
locator = Nominatim(user_agent='my_request')
location = locator.geocode('højlundsparken 71, fastrup 8355 solbjerg')

print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))



Latitude = 56.038386, Longitude = 10.069766


In [12]:
df = pd.read_csv('addresses.csv')

df["ADDRESS"] = df['Address1'].astype(str) + ", " + df['Address3'].astype(str) + "," + df['Address4'].astype(str) + "," + df['Address5'].astype(str) + ", " + "Sweden"

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df = df.drop(['Address1', 'Address3', 'Address4', 'Address5','Telefon', 'ADDRESS', 'location', 'point','Namn'], axis=1)


,Unnamed: 0,Typ,Nr,latitude,longitude,altitude
0,0,Butik,102,59.338877,18.090865,0.0
1,1,Butik,104,59.337207,18.079098,0.0
2,2,Butik,106,59.335673,18.099179,0.0
3,3,Butik,110,59.334327,18.062604,0.0
4,4,Butik,113,59.333613,18.061674,0.0


In [44]:
#df1 = df1.drop(['ADDRESS', 'Address3', 'Address4', 'Address5','Telefon', 'ADDRESS', 'location', 'point','Namn'], axis=1)
"""df=df.dropna()
df1 = pd.read_csv('updated_geo.csv')
df1 = df1.loc[:, ['longitude', 'latitude']]
df1 = df1.dropna()"""

df1['longitude'] = pd.to_numeric(df1['longitude'], errors='coerce')
df1['latitude'] = pd.to_numeric(df1['latitude'], errors='coerce')
df1 = df1.dropna()
#print(df1)
#df1.astype({'longitude': 'int32'}).dtypes


"""map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)
df1.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1
"""


# for speed purposes
MAX_RECORDS = 1000

# create empty map zoomed in on San Francisco
_map = folium.Map(location=[59.338315,18.089960], zoom_start=12)

# add a marker for every record in the filtered data, use a clustered view
for each in df1[0:MAX_RECORDS].iterrows():
    _map.simple_marker(
        location = [each[1]["Y"],each[1]["X"]],
        clustered_marker = True)

display(_map)

C:\Users\Marti\AppData\Local\Temp\ipykernel_9348\3680853424.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['longitude'] = pd.to_numeric(df1['longitude'], errors='coerce')
C:\Users\Marti\AppData\Local\Temp\ipykernel_9348\3680853424.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['latitude'] = pd.to_numeric(df1['latitude'], errors='coerce')


NameError: name 'FastMarkerCluster' is not defined